In [1]:
import csv
from os import listdir
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from numpy.random import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
with open("real1.csv", 'r', newline='') as f:
    data = csv.reader(f, quoting=csv.QUOTE_NONNUMERIC)
    l = list(data)
    print(type(l[0][0]))
    print(type(l[1][0]))

<class 'str'>
<class 'float'>


In [4]:
data = np.array(l[1:])
shuffle(data)
x = data[:, 3:-1]
y = data[:, -1]

In [50]:
x += abs(np.min(x))
x -= x.mean(axis=0)
x /= x.std(axis=0)
print(x.shape, y.shape)
print(len(x))

(847, 28) (847,)
847


In [54]:
model = Sequential()
model.add(Dense(8, activation='relu', input_shape=(np.shape(x)[1],)))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.input_shape)
print(model.output_shape)

(None, 28)
(None, 1)


In [61]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(x=x, y=y, epochs=500)
# model.save(f"../../models/v{len(listdir('../../models/'))+1}.h5")

# print(data[1])
# print(x.shape)

Epoch 1/500
27/27 [==============================] - 1s 1ms/step - loss: 0.0788 - accuracy: 0.9764
Epoch 2/500
27/27 [==============================] - 0s 2ms/step - loss: 0.0767 - accuracy: 0.9764
Epoch 3/500
27/27 [==============================] - 0s 2ms/step - loss: 0.0763 - accuracy: 0.9764
Epoch 4/500
27/27 [==============================] - 0s 2ms/step - loss: 0.0756 - accuracy: 0.9764
Epoch 5/500
27/27 [==============================] - 0s 2ms/step - loss: 0.0751 - accuracy: 0.9764
Epoch 6/500
27/27 [==============================] - 0s 2ms/step - loss: 0.0748 - accuracy: 0.9764
Epoch 7/500
27/27 [==============================] - 0s 2ms/step - loss: 0.0743 - accuracy: 0.9764
Epoch 8/500
27/27 [==============================] - 0s 2ms/step - loss: 0.0741 - accuracy: 0.9764
Epoch 9/500
27/27 [==============================] - 0s 2ms/step - loss: 0.0734 - accuracy: 0.9764
Epoch 10/500
27/27 [==============================] - 0s 2ms/step - loss: 0.0733 - accuracy: 0.9764
Epoch 11/

In [62]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
lite_model = converter.convert()
lite_model_file_name = f"../../models/v{len(listdir('../../models/'))}.tflite"
with open(lite_model_file_name, 'wb') as lite_model_file:
    lite_model_file.write(lite_model)

INFO:tensorflow:Assets written to: C:\Users\ABDUL-~1\AppData\Local\Temp\tmp1l0a6my3\assets


INFO:tensorflow:Assets written to: C:\Users\ABDUL-~1\AppData\Local\Temp\tmp1l0a6my3\assets


In [64]:
shuffle(data)
x = data[:, 3:-1]
y = data[:, -1]

prediction = model.predict(x)
print(prediction[:5].round())
print(y[:5])
print(f"accuracy: {accuracy_score(y, prediction.round())}")
print(f"precision: {precision_score(y, prediction.round())}")
print(f"recall: {recall_score(y, prediction.round())}")
print(f"f score: {f1_score(y, prediction.round())}")
prediction = prediction.round()
print("a  p")
for i in range(len(y)):
    if y[i] != prediction[i]: print(f"{y[i]}  {prediction[i]}")

27/27 [==============================] - 0s 1ms/step
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[0. 0. 0. 0. 0.]
accuracy: 1.0
precision: 1.0
recall: 1.0
f score: 1.0
a  p


In [65]:
interpreter = tf.lite.Interpreter(model_path = lite_model_file_name)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input shape:", input_details[0]["shape"])
print("Input Type:", input_details[0]["dtype"])
print("Output shape:", output_details[0]["shape"])
print("Output Type:", output_details[0]["dtype"])

Input shape: [ 1 28]
Input Type: <class 'numpy.float32'>
Output shape: [1 1]
Output Type: <class 'numpy.float32'>
